https://horejsek.github.io/python-fastjsonschema/

### setup
`pip install fastjsonschema`


### docs
https://json-schema.org/understanding-json-schema/about.html

In [1]:
import fastjsonschema as fjs

In [2]:
def is_valid(schema, data):
    try:
        validate = fjs.compile(schema)
        return validate(data) == data, None
    except fjs.JsonSchemaException as ex:
        return False, ex

#### compile() is faster

In [3]:
%time fjs.validate({'type': 'string'}, 'hello')

CPU times: user 568 µs, sys: 53 µs, total: 621 µs
Wall time: 626 µs


'hello'

In [4]:
%time fjs.compile({'type': 'string'})('hello')

CPU times: user 783 µs, sys: 0 ns, total: 783 µs
Wall time: 796 µs


'hello'

### basic type

In [5]:
schema = {'type': 'string'}
data = 'hello'

In [6]:
is_valid(schema, data)

(True, None)

In [7]:
schema = {
    'type': 'object',
    'properties': {
        'a': {'type': 'number', 'default': 42},
    },
}
data = {'a': 42}

In [8]:
is_valid(schema, data)

(True, None)

In [9]:
schema = {
    'type': 'object',
    'properties': {
        'a': {'type': 'number', 'default': 42},
    },
}
data = {}

In [10]:
is_valid(schema, data)

(True, None)

In [11]:
schema = {
    '$schema': 'http://json-schema.org/draft-04/schema',
    'type': 'number',
}
data = 100

In [12]:
is_valid(schema, data)

(True, None)

In [13]:
schema = {
    '$schema': 'http://json-schema.org/draft-04/schema',
    'type': 'number',
}
data = 'abc'

In [14]:
is_valid(schema, data)

(False, fastjsonschema.exceptions.JsonSchemaException('data must be number'))

In [52]:
schema = {
    '$schema': 'http://json-schema.org/draft-04/schema',
   "type": "string",
   "pattern": "^(\\([0-9]{3}\\))?[0-9]{3}-[0-9]{4}$",
}

data = [
    "555-1212",  "(888)555-1212" , "(888)555-1212 ext. 532"
]

for d in data:
    print(is_valid(schema, d))

(True, None)
(True, None)
(False, JsonSchemaException('data must match pattern ^(\\([0-9]{3}\\))?[0-9]{3}-[0-9]{4}$'))


In [55]:
schema = {
    '$schema': 'http://json-schema.org/draft-04/schema',
   "type": "string",
   "pattern": "^[0-9]{4}-[0-9]{2}-[0-9]{2}$",
}

data = [
    "2020-01-01",  "2020/01/01"
]

for d in data:
    print(is_valid(schema, d))

(True, None)
(False, JsonSchemaException('data must match pattern ^[0-9]{4}-[0-9]{2}-[0-9]{2}$'))


### complex schema: `nested`, `$ref`

https://json-schema.org/understanding-json-schema/structuring.html

In [15]:
schema = {
  "$schema": "http://json-schema.org/draft-07/schema#",

  "definitions": {
    "address": {
      "type": "object",
      "properties": {
        "street_address": { "type": "string" },
        "city":           { "type": "string" },
        "state":          { "type": "string" }
      },
      "required": ["street_address", "city", "state"]
    }
  },

  "type": "object",

  "properties": {
    "name": { "type": "string" },
    "billing_address": { "$ref": "#/definitions/address" },
    "shipping_address": { "$ref": "#/definitions/address" }
  },
  "required": ["name", "billing_address"],
}

data = {
  "name": "Jose Goodman",
  "shipping_address": {
    "street_address": "1600 Pennsylvania Avenue NW",
    "city": "Washington",
    "state": "DC"
  },
  "billing_address": {
    "street_address": "1st Street SE",
    "city": "Washington",
    "state": "DC"
  }
}

In [16]:
is_valid(schema, data)

(True, None)

In [56]:
schema = {
    "$schema": "http://json-schema.org/draft-07/schema#",
  "type": "object",
  "properties": {
    "first_name": { "type": "string" },
    "last_name": { "type": "string" },
    "birthday": {    
            "type": "string",
           "pattern": "^[0-9]{4}-[0-9]{2}-[0-9]{2}$",
        },
    "address": {
      "type": "object",
      "properties": {
        "street_address": { "type": "string" },
        "city": { "type": "string" },
        "state": { "type": "string" },
        "country": { "type" : "string" },
      },
    },
  },
  "required" : ["first_name", "last_name", "address"],
}

data = [
    {
      "name": "George Washington",
      "birthday": "February 22, 1732",
      "address": "Mount Vernon, Virginia, United States",
    },    
    {
      "first_name": "George",
      "last_name": "Washington",
      "birthday": "2000/01/01",
      "address": {
        "street_address": "3200 Mount Vernon Memorial Highway",
        "city": "Mount Vernon",
        "state": "Virginia",
        "country": "United States",
      }
    },    
    {
      "first_name": "George",
      "last_name": "Washington",
      "birthday": "2000-01-01",
      "address": {
        "street_address": "3200 Mount Vernon Memorial Highway",
        "city": "Mount Vernon",
        "state": "Virginia",
        "country": "United States",
      }
    }    
]

In [57]:
for d in data:
    print(is_valid(schema, d))

(False, JsonSchemaException("data must contain ['first_name', 'last_name', 'address'] properties"))
(False, JsonSchemaException('data.birthday must match pattern ^[0-9]{4}-[0-9]{2}-[0-9]{2}$'))
(True, None)


### conditional

https://json-schema.org/understanding-json-schema/reference/conditionals.html

In [17]:
schema = {
  "type": "object",
  "properties": {
    "street_address": {
      "type": "string"
    },
    "country": {
      "enum": ["United States of America", "Canada"]
    }
  },
  "if": {
    "properties": { "country": { "const": "United States of America" } }
  },
  "then": {
    "properties": { "postal_code": { "pattern": "[0-9]{5}(-[0-9]{4})?" } }
  },
  "else": {
    "properties": { "postal_code": { "pattern": "[A-Z][0-9][A-Z] [0-9][A-Z][0-9]" } }
  }
}

In [18]:
data = [
    {
      "street_address": "1600 Pennsylvania Avenue NW",
      "country": "United States of America",
      "postal_code": "20500"
    },
    {
      "street_address": "24 Sussex Drive",
      "country": "Canada",
      "postal_code": "K1M 1M4"
    },    
    {
      "street_address": "24 Sussex Drive",
      "country": "Canada",
      "postal_code": "10000"
    }
]

In [20]:
for d in data:
    print(is_valid(schema, d))

(True, None)
(True, None)
(False, JsonSchemaException('data.postal_code must match pattern [A-Z][0-9][A-Z] [0-9][A-Z][0-9]'))
